In [54]:
import numpy as np
import pandas as pd
import math
import random

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# preprocessing
import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, learning_curve, ShuffleSplit
from sklearn.model_selection import cross_val_predict as cvp
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, confusion_matrix, explained_variance_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel, SelectKBest, RFE, chi2
from sklearn.neighbors import KNeighborsClassifier

# models
from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier, LassoCV
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

## IMPORT DATASET

In [2]:
df1 = pd.read_csv("../data/chipVariantCalling_run1.tsv", sep='\t')
df2 = pd.read_csv("../data/chipVariantCalling_run2.tsv", sep='\t')

df1

,d.barcode,DP,VD,AF,HIAF,IMPACT,SYMBOL,loci,sampleTimePt,gender,MSID,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl
0,4010289633,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,MS2083,2:2,3644:3596,12:14,34.0,1.182210,CHIP
1,4010289633,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,MS2083,2:2,3644:3596,12:14,34.0,1.182210,CHIP
2,4010289633,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,MS2083,2:2,3644:3596,12:14,34.0,1.182210,CHIP
3,4010289633,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,MS2083,2:2,3644:3596,12:14,34.0,1.182210,CHIP
4,4010289633,7282,29,0.0040,0.0036,MODERATE,GNB1,chr1:1747250_T/C,Baseline,Male,MS2083,2:2,3639:3606,15:14,32.7,1.061729,CHIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137944,4010290016,731,2,0.0027,0.0028,MODERATE,ZFY,chrY:2848011_C/T,Blank,Blank,NaN,2:2,371:357,1:1,31.0,1.039170,NaN
1137945,4010290016,731,2,0.0027,0.0028,MODERATE,ZFY,chrY:2848029_C/T,Blank,Blank,NaN,2:2,367:359,1:1,37.0,1.022260,NaN
1137946,4010290016,731,2,0.0027,0.0028,MODERATE,ZFY,chrY:2848029_C/T,Blank,Blank,NaN,2:2,367:359,1:1,37.0,1.022260,NaN
1137947,4010290016,731,2,0.0027,0.0028,MODERATE,ZFY,chrY:2848029_C/T,Blank,Blank,NaN,2:2,367:359,1:1,37.0,1.022260,NaN


In [3]:
unknown1 = df1[(df1.chipOrControl == "Unknown")]
unknown2 = df2[(df2.chipOrControl == "Unknown")]

In [4]:
df1 = df1[(df1.chipOrControl != "Blank") & (df1.chipOrControl != "Unknown")]
df1 = df1.dropna(subset=['chipOrControl'])
df1.drop(['MSID', 'd.barcode'], axis=1, inplace=True)
# df.drop(['sampleTimePt'], axis = 1, inplace=True)

df2 = df2[(df2.chipOrControl != "Blank") & (df2.chipOrControl != "Unknown")]
df2 = df2.dropna(subset=['chipOrControl'])
df2.drop(['MSID', 'd.barcode'], axis=1, inplace=True)
# df2.drop(['sampleTimePt'], axis = 1, inplace=True)

In [5]:
genes = df1.SYMBOL.unique()
genes2 = df2.SYMBOL.unique()

In [6]:
df1 = df1[(df1.BIAS == "2:2")]
df1


,DP,VD,AF,HIAF,IMPACT,SYMBOL,loci,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl
0,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,2:2,3644:3596,12:14,34.0,1.182210,CHIP
1,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,2:2,3644:3596,12:14,34.0,1.182210,CHIP
2,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,2:2,3644:3596,12:14,34.0,1.182210,CHIP
3,7281,26,0.0036,0.0033,MODERATE,GNB1,chr1:1747196_T/C,Baseline,Male,2:2,3644:3596,12:14,34.0,1.182210,CHIP
4,7282,29,0.0040,0.0036,MODERATE,GNB1,chr1:1747250_T/C,Baseline,Male,2:2,3639:3606,15:14,32.7,1.061729,CHIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536743,711,2,0.0028,0.0028,MODERATE,BRCC3,chrX:154305535_C/G,Y3,Female,2:2,356:353,1:1,25.0,1.008490,Control
536744,711,2,0.0028,0.0028,MODERATE,BRCC3,chrX:154305535_C/G,Y3,Female,2:2,356:353,1:1,25.0,1.008490,Control
536745,711,2,0.0028,0.0028,MODERATE,BRCC3,chrX:154305535_C/G,Y3,Female,2:2,356:353,1:1,25.0,1.008490,Control
536746,711,2,0.0028,0.0028,MODERATE,BRCC3,chrX:154305535_C/G,Y3,Female,2:2,356:353,1:1,25.0,1.008490,Control


### Columns `BIAS`, `REFBIAS` and `VARBIAS` are strings. Change to floats.

In [7]:
def ratio_to_int(string):
    a, b = string.split(":")
    if int(b) == 0:
        return 0
    else:
        return int(a) / int(b)

In [8]:
# for dataset 1
bias = []
refbias = []
varbias = []

for ratio in df1.BIAS.array:
    bias.append(ratio_to_int(ratio)) 

for ratio in df1.REFBIAS.array:
    refbias.append(ratio_to_int(ratio)) 

for ratio in df1.VARBIAS.array:
    varbias.append(ratio_to_int(ratio)) 
    
bias = pd.Series(bias)
refbias = pd.Series(refbias)
varbias = pd.Series(varbias)

df1['BIAS'] = bias.values
df1['REFBIAS'] = refbias.values
df1['VARBIAS'] = varbias.values

In [9]:
# for dataset 2
bias = []
refbias = []
varbias = []

for ratio in df2.BIAS.array:
    bias.append(ratio_to_int(ratio)) 

for ratio in df2.REFBIAS.array:
    refbias.append(ratio_to_int(ratio)) 

for ratio in df2.VARBIAS.array:
    varbias.append(ratio_to_int(ratio)) 
    
bias = pd.Series(bias)
refbias = pd.Series(refbias)
varbias = pd.Series(varbias)

df2['BIAS'] = bias.values
df2['REFBIAS'] = refbias.values
df2['VARBIAS'] = varbias.values

In [10]:
objs = {}
lst = []
for i in df1.columns:
    if df1.dtypes[i] == object:
        if len(df1[f"{i}"].unique()) <= 100:
            objs[i] = len(df1[f"{i}"].unique())
            lst.append(i)

In [11]:
objs = {}
lst = []
for i in df2.columns:
    if df2.dtypes[i] == object:
        if len(df2[f"{i}"].unique()) <= 100:
            objs[i] = len(df2[f"{i}"].unique())
            lst.append(i)

In [12]:
for i in lst:
    k = i
    dict = {}
    df_new = df1
    for ix, i in zip(range(len(df_new[i].unique())), df_new[i].unique() ):
        dict[i] = ix
    df1 = df1.replace({f"{k}": dict})
    df1[f"{k}"] = df1[f"{k}"].astype(str).astype(float)

In [13]:
for i in lst:
    k = i
    dict = {}
    df2_new = df2
    for ix, i in zip(range(len(df2_new[i].unique())), df2_new[i].unique() ):
        dict[i] = ix
    df2 = df2.replace({f"{k}": dict})
    df2[f"{k}"] = df2[f"{k}"].astype(str).astype(float)

In [14]:
# LOCI not required in final model
df1.drop(['loci'], axis=1, inplace=True)
df2.drop(['loci'], axis=1, inplace=True)

In [15]:
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

### `REFBIAS` Outliers

In [16]:
df1['REFBIAS_naturalLog'] = np.log(df1['REFBIAS'])
df2['REFBIAS_naturalLog'] = np.log(df2['REFBIAS'])
df1

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog
0,7281,26,0.0036,0.0033,0.0,0.0,0.0,0.0,1.0,1.013348,0.857143,34.0,1.182210,0.0,0.013260
4,7282,29,0.0040,0.0036,0.0,0.0,0.0,0.0,1.0,1.009151,1.071429,32.7,1.061729,0.0,0.009110
8,7282,24,0.0033,0.0033,0.0,0.0,0.0,0.0,1.0,1.005268,1.000000,34.9,1.005270,0.0,0.005254
12,178,2,0.0112,0.0120,0.0,1.0,0.0,0.0,1.0,1.000000,1.000000,37.0,1.000000,0.0,0.000000
13,1773,6,0.0034,0.0035,0.0,1.0,0.0,0.0,1.0,1.017162,1.000000,37.0,1.017150,0.0,0.017017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536717,711,4,0.0056,0.0057,0.0,28.0,1.0,1.0,1.0,1.008523,1.000000,37.0,1.008510,1.0,0.008487
536725,711,2,0.0028,0.0028,0.0,28.0,1.0,1.0,1.0,1.008499,1.000000,31.0,1.008490,1.0,0.008463
536733,711,2,0.0028,0.0029,1.0,28.0,1.0,1.0,1.0,1.005666,1.000000,37.0,1.005660,1.0,0.005650
536734,711,2,0.0028,0.0029,0.0,28.0,1.0,1.0,1.0,1.008499,1.000000,37.0,1.008490,1.0,0.008463


In [17]:
df1.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog
count,106255.000000,106255.000000,106255.000000,106255.000000,106255.000000,106255.000000,106255.000000,106255.000000,106255.0,106255.000000,106255.000000,106255.000000,106255.000000,106255.000000,106255.000000
mean,5885.967503,143.357922,0.009654,0.009677,0.053014,12.247217,0.525029,0.481747,1.0,1.021461,1.021984,35.287810,1.108922,0.446115,0.020584
std,13447.104233,3009.024829,0.056847,0.057190,0.224062,7.965296,0.499375,0.499669,0.0,0.026883,0.360202,2.619363,0.424406,0.497090,0.044277
min,4.000000,2.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,1.0,0.060000,0.075269,22.500000,1.000000,0.000000,-2.813411
25%,555.000000,2.000000,0.002800,0.002800,0.000000,7.000000,0.000000,0.000000,1.0,1.013369,1.000000,34.400000,1.014460,0.000000,0.013280
50%,1214.000000,4.000000,0.003500,0.003500,0.000000,9.000000,1.000000,0.000000,1.0,1.020408,1.000000,37.000000,1.023910,0.000000,0.020203
75%,5214.000000,16.000000,0.005200,0.005200,0.000000,20.000000,1.000000,1.000000,1.0,1.028807,1.000000,37.000000,1.042621,1.000000,0.028399
max,455487.000000,319533.000000,0.996600,0.998200,1.000000,38.000000,1.000000,1.000000,1.0,2.000000,18.000000,37.000000,17.780939,1.000000,0.693147


In [18]:
# With dataset 1
#upper
print("Above the median quantile")
print(df1["REFBIAS"].quantile(0.97))
print(df1["REFBIAS"].quantile(0.98))
print(df1["REFBIAS"].quantile(0.99))
print()
print(df1["REFBIAS_naturalLog"].quantile(0.97))
print(df1["REFBIAS_naturalLog"].quantile(0.98))
print(df1["REFBIAS_naturalLog"].quantile(0.99))
print()
#lower
print("Below the median quantile")
print(df1["REFBIAS"].quantile(0.1))
print(df1["REFBIAS"].quantile(0.05))
print(df1["REFBIAS"].quantile(0.01))
print()
print(df1["REFBIAS_naturalLog"].quantile(0.1))
print(df1["REFBIAS_naturalLog"].quantile(0.05))
print(df1["REFBIAS_naturalLog"].quantile(0.01))

Above the median quantile
1.0526315789473684
1.05792656270825
1.0692444646410828

0.05129329438755048
0.056310919597782616
0.06695229115391255

Below the median quantile
1.0068493150684932
1.0
0.989247311827957

0.006825965070399891
0.0
-0.010810916104215617


In [19]:
#upper cut off at 0.067?
q = df1["REFBIAS_naturalLog"].quantile(0.98)
df1 = df1[df1["REFBIAS_naturalLog"] < q]
#lower cut off at 0.0038
q = df1["REFBIAS_naturalLog"].quantile(0.10)
df1 = df1[df1["REFBIAS_naturalLog"] > q]
df1.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog
count,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.0,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000
mean,6419.087733,83.721562,0.006386,0.006359,0.051395,12.177408,0.519054,0.481404,1.0,1.023237,1.018297,35.249122,1.106227,0.441865,0.022920
std,13924.157559,1715.182473,0.030187,0.030266,0.220804,7.856957,0.499639,0.499657,0.0,0.010385,0.323537,2.600519,0.375312,0.496612,0.010109
min,39.000000,2.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,1.0,1.006735,0.075269,22.500000,1.000000,0.000000,0.006712
25%,618.000000,2.000000,0.002800,0.002800,0.000000,7.000000,0.000000,0.000000,1.0,1.015408,1.000000,34.400000,1.016420,0.000000,0.015291
50%,1412.000000,6.000000,0.003400,0.003300,0.000000,9.000000,1.000000,0.000000,1.0,1.021490,1.000000,36.900000,1.025040,0.000000,0.021262
75%,6047.000000,19.000000,0.004800,0.004800,0.000000,20.000000,1.000000,1.000000,1.0,1.029246,1.000000,37.000000,1.042500,1.000000,0.028826
max,455487.000000,137339.000000,0.988600,0.994000,1.000000,38.000000,1.000000,1.000000,1.0,1.057923,18.000000,37.000000,17.780939,1.000000,0.056308


In [20]:
df2.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog
count,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,157033.000000,1.570330e+05
mean,2993.601905,53.156107,0.011415,0.011345,0.956168,14.827851,0.525234,0.496138,0.849844,9.747043,1.542255,32.155388,2.357794,0.478237,-inf
std,5145.917259,1076.017675,0.057943,0.059323,0.204721,9.381112,0.499364,0.499987,0.376606,160.293513,4.741635,4.414058,18.181966,0.499528,NaN
min,2.000000,2.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.500000,0.000000,0.000000,-inf
25%,729.000000,3.000000,0.002900,0.002800,1.000000,8.000000,0.000000,0.000000,1.000000,0.862069,0.500000,28.300000,1.016570,0.000000,-1.484200e-01
50%,1567.000000,6.000000,0.003700,0.003700,1.000000,13.000000,1.000000,0.000000,1.000000,0.979557,1.000000,33.000000,1.100140,0.000000,-2.065478e-02
75%,3331.000000,13.000000,0.006000,0.005800,1.000000,23.000000,1.000000,1.000000,1.000000,1.026915,1.000000,37.000000,1.652000,1.000000,2.655927e-02
max,216036.000000,173172.000000,1.000000,1.000000,1.000000,40.000000,1.000000,1.000000,2.000000,13870.000000,575.000000,37.000000,3571.428571,1.000000,9.537484e+00


In [21]:
# With dataset 2
#upper
print("Above the median quantile")
print(df2["REFBIAS"].quantile(0.97))
print(df2["REFBIAS"].quantile(0.98))
print(df2["REFBIAS"].quantile(0.99))
print()
print(df2["REFBIAS_naturalLog"].quantile(0.97))
print(df2["REFBIAS_naturalLog"].quantile(0.98))
print(df2["REFBIAS_naturalLog"].quantile(0.99))
print()
#lower
print("Below the median quantile")
print(df2["REFBIAS"].quantile(0.1))
print(df2["REFBIAS"].quantile(0.05))
print(df2["REFBIAS"].quantile(0.01))
print()
print(df2["REFBIAS_naturalLog"].quantile(0.6))
print(df2["REFBIAS_naturalLog"].quantile(0.05))
print(df2["REFBIAS_naturalLog"].quantile(0.01))

Above the median quantile
1.1111111111111112
1.1422222626258425
12.266153846148743

0.10536051565782635
0.1329757179848386
2.5064518185870805

Below the median quantile
0.5623794086487293
0.23593930482230804
0.0

0.0009696972738023772
-1.4441806907359855
-inf


In [22]:
#upper cut off at 0.142?
q = df2["REFBIAS_naturalLog"].quantile(0.98)
df2 = df2[df2["REFBIAS_naturalLog"] < q]
#lower cut off at -1.444
q = df2["REFBIAS_naturalLog"].quantile(0.05)
df2 = df2[df2["REFBIAS_naturalLog"] > q]
df2.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog
count,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000,146197.000000
mean,3055.650232,36.680301,0.009267,0.009070,0.955649,14.844710,0.525072,0.496679,0.887576,0.928065,1.624989,32.122261,2.176161,0.478006,-0.095720
std,5203.571381,612.635943,0.041306,0.042181,0.205875,9.441809,0.499373,0.499991,0.337901,0.159505,4.729975,4.401410,8.748302,0.499518,0.226791
min,4.000000,2.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,0.000000,0.226827,0.000000,22.500000,0.000000,0.000000,-1.483569
25%,738.000000,3.000000,0.002900,0.002800,1.000000,8.000000,0.000000,0.000000,1.000000,0.885196,0.666667,28.300000,1.023720,0.000000,-0.121946
50%,1601.000000,6.000000,0.003700,0.003700,1.000000,13.000000,1.000000,0.000000,1.000000,0.983389,1.000000,32.800000,1.114650,0.000000,-0.016751
75%,3435.000000,13.000000,0.006000,0.005800,1.000000,23.000000,1.000000,1.000000,1.000000,1.026415,1.083333,37.000000,1.685573,1.000000,0.026072
max,216036.000000,69194.000000,0.986900,1.000000,1.000000,40.000000,1.000000,1.000000,2.000000,1.142212,575.000000,37.000000,1234.567901,1.000000,0.132966


## `DP` AND `VD` OUTLIERS

In [23]:
(df1.describe())

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog
count,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.0,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000,93705.000000
mean,6419.087733,83.721562,0.006386,0.006359,0.051395,12.177408,0.519054,0.481404,1.0,1.023237,1.018297,35.249122,1.106227,0.441865,0.022920
std,13924.157559,1715.182473,0.030187,0.030266,0.220804,7.856957,0.499639,0.499657,0.0,0.010385,0.323537,2.600519,0.375312,0.496612,0.010109
min,39.000000,2.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,1.0,1.006735,0.075269,22.500000,1.000000,0.000000,0.006712
25%,618.000000,2.000000,0.002800,0.002800,0.000000,7.000000,0.000000,0.000000,1.0,1.015408,1.000000,34.400000,1.016420,0.000000,0.015291
50%,1412.000000,6.000000,0.003400,0.003300,0.000000,9.000000,1.000000,0.000000,1.0,1.021490,1.000000,36.900000,1.025040,0.000000,0.021262
75%,6047.000000,19.000000,0.004800,0.004800,0.000000,20.000000,1.000000,1.000000,1.0,1.029246,1.000000,37.000000,1.042500,1.000000,0.028826
max,455487.000000,137339.000000,0.988600,0.994000,1.000000,38.000000,1.000000,1.000000,1.0,1.057923,18.000000,37.000000,17.780939,1.000000,0.056308


In [24]:
print("DATASET 1")
print("DEPTH")
print("DP at 97% quantile: " + str(df1.DP.quantile(0.97)))
print("DP at 98% quantile: " + str(df1.DP.quantile(0.98)))
print("DP at 99% quantile: " + str(df1.DP.quantile(0.99)))
print("VARIANT DEPTH")
print("VD at 97% quantile: " + str(df1.VD.quantile(0.97)))
print("VD at 98% quantile: " + str(df1.VD.quantile(0.98)))
print("VD at 99% quantile: " + str(df1.VD.quantile(0.99)))

print()

print("DATASET 2")
print("DEPTH")
print("DP at 97% quantile: " + str(df2.DP.quantile(0.97)))
print("DP at 98% quantile: " + str(df2.DP.quantile(0.98)))
print("DP at 99% quantile: " + str(df2.DP.quantile(0.99)))
print("VARIANT DEPTH")
print("VD at 97% quantile: " + str(df2.VD.quantile(0.97)))
print("VD at 98% quantile: " + str(df2.VD.quantile(0.98)))
print("VD at 99% quantile: " + str(df2.VD.quantile(0.99)))

DATASET 1
DEPTH
DP at 97% quantile: 37905.76000000001
DP at 98% quantile: 47790.47999999997
DP at 99% quantile: 66874.96
VARIANT DEPTH
VD at 97% quantile: 123.0
VD at 98% quantile: 159.0
VD at 99% quantile: 235.0

DATASET 2
DEPTH
DP at 97% quantile: 13154.0
DP at 98% quantile: 16047.159999999974
DP at 99% quantile: 21929.520000000106
VARIANT DEPTH
VD at 97% quantile: 92.0
VD at 98% quantile: 135.0
VD at 99% quantile: 266.0


In [25]:
df1['VD_naturalLog'] = np.log(df1['VD'])
df2['VD_naturalLog'] = np.log(df2['VD'])
df1

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog,VD_naturalLog
0,7281,26,0.0036,0.0033,0.0,0.0,0.0,0.0,1.0,1.013348,0.857143,34.0,1.182210,0.0,0.013260,3.258097
4,7282,29,0.0040,0.0036,0.0,0.0,0.0,0.0,1.0,1.009151,1.071429,32.7,1.061729,0.0,0.009110,3.367296
13,1773,6,0.0034,0.0035,0.0,1.0,0.0,0.0,1.0,1.017162,1.000000,37.0,1.017150,0.0,0.017017,1.791759
14,1773,7,0.0039,0.0029,0.0,1.0,0.0,0.0,1.0,1.020619,0.750000,29.6,1.360580,0.0,0.020409,1.945910
15,1774,8,0.0045,0.0046,0.0,1.0,0.0,0.0,1.0,1.014840,1.000000,35.5,1.014830,0.0,0.014731,2.079442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536693,710,4,0.0056,0.0043,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,30.5,1.014270,1.0,0.014185,1.386294
536717,711,4,0.0056,0.0057,0.0,28.0,1.0,1.0,1.0,1.008523,1.000000,37.0,1.008510,1.0,0.008487,1.386294
536725,711,2,0.0028,0.0028,0.0,28.0,1.0,1.0,1.0,1.008499,1.000000,31.0,1.008490,1.0,0.008463,0.693147
536734,711,2,0.0028,0.0029,0.0,28.0,1.0,1.0,1.0,1.008499,1.000000,37.0,1.008490,1.0,0.008463,0.693147


In [26]:
# With dataset 1
#upper
print("Above the median quantile")
print(df1.VD_naturalLog.quantile(0.97))
print(df1.VD_naturalLog.quantile(0.98))
print(df1.VD_naturalLog.quantile(0.99))
print()
print(df1.VD.quantile(0.97))
print(df1.VD.quantile(0.98))
print(df1.VD.quantile(0.99))

print()

#Lower
print("Below the median quantile")
print(df1.VD_naturalLog.quantile(0.1))
print(df1.VD_naturalLog.quantile(0.05))
print(df1.VD_naturalLog.quantile(0.01))
print()
print(df1.VD.quantile(0.1))
print(df1.VD.quantile(0.05))
print(df1.VD.quantile(0.01))

Above the median quantile
4.812184355372417
5.0689042022202315
5.459585514144159

123.0
159.0
235.0

Below the median quantile
0.6931471805599453
0.6931471805599453
0.6931471805599453

2.0
2.0
2.0


In [27]:
#cut off at 4.779?
q = df1["VD_naturalLog"].quantile(0.97)
df1 = df1[df1["VD_naturalLog"] < q]

q = df1["VD_naturalLog"].quantile(0.05)
df1 = df1[df1["VD_naturalLog"] > q]
df1.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog,VD_naturalLog
count,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.0,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000
mean,6455.318728,20.025384,0.004264,0.004154,0.037594,12.337813,0.494051,0.483101,1.0,1.022656,1.026938,34.879164,1.140353,0.435615,0.022358,2.444376
std,8289.446202,23.572148,0.006335,0.006418,0.190213,7.864064,0.499968,0.499718,0.0,0.009697,0.383866,2.676289,0.441883,0.495841,0.009445,1.017194
min,41.000000,3.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,1.0,1.006735,0.075269,22.500000,1.000000,0.000000,0.006712,1.098612
25%,1193.000000,4.000000,0.002700,0.002700,0.000000,7.000000,0.000000,0.000000,1.0,1.015464,1.000000,34.000000,1.017270,0.000000,0.015346,1.386294
50%,2630.000000,9.000000,0.003200,0.003100,0.000000,10.000000,0.000000,0.000000,1.0,1.021127,1.000000,35.700000,1.026410,0.000000,0.020907,2.197225
75%,8553.000000,26.000000,0.004300,0.004200,0.000000,20.000000,1.000000,1.000000,1.0,1.028146,1.000000,37.000000,1.066280,1.000000,0.027757,3.258097
max,49697.000000,122.000000,0.643400,0.643400,1.000000,38.000000,1.000000,1.000000,1.0,1.057923,18.000000,37.000000,17.780939,1.000000,0.056308,4.804021


In [28]:
# With dataset 2
#upper
print("Above the median quantile")
print(df2.VD_naturalLog.quantile(0.97))
print(df2.VD_naturalLog.quantile(0.98))
print(df2.VD_naturalLog.quantile(0.99))
print()
print(df2.VD.quantile(0.97))
print(df2.VD.quantile(0.98))
print(df2.VD.quantile(0.99))

print()

#Lower
print("Below the median quantile")
print(df2.VD_naturalLog.quantile(0.1))
print(df2.VD_naturalLog.quantile(0.05))
print(df2.VD_naturalLog.quantile(0.01))
print()
print(df2.VD.quantile(0.1))
print(df2.VD.quantile(0.05))
print(df2.VD.quantile(0.01))

Above the median quantile
4.5217885770490405
4.90527477843843
5.583496308781699

92.0
135.0
266.0

Below the median quantile
0.6931471805599453
0.6931471805599453
0.6931471805599453

2.0
2.0
2.0


In [29]:
#cut off at 4.522?
q = df2["VD_naturalLog"].quantile(0.97)
df2 = df2[df2["VD_naturalLog"] < q]

q = df2["VD_naturalLog"].quantile(0.05)
df2 = df2[df2["VD_naturalLog"] > q]
df2.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog,VD_naturalLog
count,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000,141798.000000
mean,2658.359159,10.937150,0.006603,0.006450,0.954872,14.794588,0.527878,0.495479,0.883327,0.927995,1.398141,32.239969,1.851460,0.479189,-0.095913,1.922337
std,3289.263874,13.154869,0.015518,0.016265,0.207585,9.460895,0.499224,0.499981,0.341139,0.159824,2.257818,4.353255,2.950377,0.499568,0.227515,0.923404
min,4.000000,2.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,0.000000,0.226827,0.000000,22.500000,0.000000,0.000000,-1.483569,0.693147
25%,720.000000,3.000000,0.002900,0.002800,1.000000,8.000000,0.000000,0.000000,1.000000,0.885400,0.666667,28.300000,1.022674,0.000000,-0.121716,1.098612
50%,1550.000000,6.000000,0.003700,0.003600,1.000000,13.000000,1.000000,0.000000,1.000000,0.983506,1.000000,33.000000,1.107119,0.000000,-0.016631,1.791759
75%,3219.000000,12.000000,0.005600,0.005500,1.000000,23.000000,1.000000,1.000000,1.000000,1.026688,1.000000,37.000000,1.603200,1.000000,0.026338,2.484907
max,36590.000000,91.000000,0.813600,0.872700,1.000000,40.000000,1.000000,1.000000,2.000000,1.142212,86.000000,37.000000,173.913043,1.000000,0.132966,4.510860


In [30]:
df1['QUAL_naturalLog'] = np.log(df1['QUAL'])
df2['QUAL_naturalLog'] = np.log(df2['QUAL'])
df1

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog,VD_naturalLog,QUAL_naturalLog
0,7281,26,0.0036,0.0033,0.0,0.0,0.0,0.0,1.0,1.013348,0.857143,34.0,1.182210,0.0,0.013260,3.258097,3.526361
4,7282,29,0.0040,0.0036,0.0,0.0,0.0,0.0,1.0,1.009151,1.071429,32.7,1.061729,0.0,0.009110,3.367296,3.487375
13,1773,6,0.0034,0.0035,0.0,1.0,0.0,0.0,1.0,1.017162,1.000000,37.0,1.017150,0.0,0.017017,1.791759,3.610918
14,1773,7,0.0039,0.0029,0.0,1.0,0.0,0.0,1.0,1.020619,0.750000,29.6,1.360580,0.0,0.020409,1.945910,3.387774
15,1774,8,0.0045,0.0046,0.0,1.0,0.0,0.0,1.0,1.014840,1.000000,35.5,1.014830,0.0,0.014731,2.079442,3.569533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536636,1698,6,0.0035,0.0036,0.0,28.0,1.0,1.0,1.0,1.026442,1.000000,35.0,1.026430,1.0,0.026099,1.791759,3.555348
536645,710,6,0.0085,0.0086,0.0,28.0,1.0,1.0,1.0,1.011429,1.000000,37.0,1.011410,1.0,0.011364,1.791759,3.610918
536660,710,4,0.0056,0.0057,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,34.0,1.014270,1.0,0.014185,1.386294,3.526361
536693,710,4,0.0056,0.0043,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,30.5,1.014270,1.0,0.014185,1.386294,3.417727


In [31]:
df1.describe()

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog,VD_naturalLog,QUAL_naturalLog
count,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.0,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000,64293.000000
mean,6455.318728,20.025384,0.004264,0.004154,0.037594,12.337813,0.494051,0.483101,1.0,1.022656,1.026938,34.879164,1.140353,0.435615,0.022358,2.444376,3.548647
std,8289.446202,23.572148,0.006335,0.006418,0.190213,7.864064,0.499968,0.499718,0.0,0.009697,0.383866,2.676289,0.441883,0.495841,0.009445,1.017194,0.082570
min,41.000000,3.000000,0.002500,0.001500,0.000000,0.000000,0.000000,0.000000,1.0,1.006735,0.075269,22.500000,1.000000,0.000000,0.006712,1.098612,3.113515
25%,1193.000000,4.000000,0.002700,0.002700,0.000000,7.000000,0.000000,0.000000,1.0,1.015464,1.000000,34.000000,1.017270,0.000000,0.015346,1.386294,3.526361
50%,2630.000000,9.000000,0.003200,0.003100,0.000000,10.000000,0.000000,0.000000,1.0,1.021127,1.000000,35.700000,1.026410,0.000000,0.020907,2.197225,3.575151
75%,8553.000000,26.000000,0.004300,0.004200,0.000000,20.000000,1.000000,1.000000,1.0,1.028146,1.000000,37.000000,1.066280,1.000000,0.027757,3.258097,3.610918
max,49697.000000,122.000000,0.643400,0.643400,1.000000,38.000000,1.000000,1.000000,1.0,1.057923,18.000000,37.000000,17.780939,1.000000,0.056308,4.804021,3.610918


In [32]:
# With dataset 1
#upper
print("Above the median quantile")
print(df1.QUAL_naturalLog.quantile(0.90))
print(df1.QUAL_naturalLog.quantile(0.98))
print(df1.QUAL_naturalLog.quantile(0.99))
print()
print(df1.QUAL.quantile(0.70))
print(df1.QUAL.quantile(0.98))
print(df1.QUAL.quantile(0.99))

print()

Above the median quantile
3.6109179126442243
3.6109179126442243
3.6109179126442243

37.0
37.0
37.0



In [33]:
df1 = df1[df1["QUAL"] >= 30]
df2 = df2[df2["QUAL"] >= 30]

In [34]:
df1

,DP,VD,AF,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl,REFBIAS_naturalLog,VD_naturalLog,QUAL_naturalLog
0,7281,26,0.0036,0.0033,0.0,0.0,0.0,0.0,1.0,1.013348,0.857143,34.0,1.182210,0.0,0.013260,3.258097,3.526361
4,7282,29,0.0040,0.0036,0.0,0.0,0.0,0.0,1.0,1.009151,1.071429,32.7,1.061729,0.0,0.009110,3.367296,3.487375
13,1773,6,0.0034,0.0035,0.0,1.0,0.0,0.0,1.0,1.017162,1.000000,37.0,1.017150,0.0,0.017017,1.791759,3.610918
15,1774,8,0.0045,0.0046,0.0,1.0,0.0,0.0,1.0,1.014840,1.000000,35.5,1.014830,0.0,0.014731,2.079442,3.569533
16,1774,10,0.0056,0.0057,0.0,1.0,0.0,0.0,1.0,1.021839,1.000000,37.0,1.021830,0.0,0.021604,2.302585,3.610918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536636,1698,6,0.0035,0.0036,0.0,28.0,1.0,1.0,1.0,1.026442,1.000000,35.0,1.026430,1.0,0.026099,1.791759,3.555348
536645,710,6,0.0085,0.0086,0.0,28.0,1.0,1.0,1.0,1.011429,1.000000,37.0,1.011410,1.0,0.011364,1.791759,3.610918
536660,710,4,0.0056,0.0057,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,34.0,1.014270,1.0,0.014185,1.386294,3.526361
536693,710,4,0.0056,0.0043,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,30.5,1.014270,1.0,0.014185,1.386294,3.417727


In [35]:
df1.drop(['REFBIAS_naturalLog','QUAL_naturalLog', 'VD_naturalLog', 'AF'], axis = 1)

,DP,VD,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl
0,7281,26,0.0033,0.0,0.0,0.0,0.0,1.0,1.013348,0.857143,34.0,1.182210,0.0
4,7282,29,0.0036,0.0,0.0,0.0,0.0,1.0,1.009151,1.071429,32.7,1.061729,0.0
13,1773,6,0.0035,0.0,1.0,0.0,0.0,1.0,1.017162,1.000000,37.0,1.017150,0.0
15,1774,8,0.0046,0.0,1.0,0.0,0.0,1.0,1.014840,1.000000,35.5,1.014830,0.0
16,1774,10,0.0057,0.0,1.0,0.0,0.0,1.0,1.021839,1.000000,37.0,1.021830,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536636,1698,6,0.0036,0.0,28.0,1.0,1.0,1.0,1.026442,1.000000,35.0,1.026430,1.0
536645,710,6,0.0086,0.0,28.0,1.0,1.0,1.0,1.011429,1.000000,37.0,1.011410,1.0
536660,710,4,0.0057,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,34.0,1.014270,1.0
536693,710,4,0.0043,0.0,28.0,1.0,1.0,1.0,1.014286,1.000000,30.5,1.014270,1.0


In [36]:
df2.drop(['REFBIAS_naturalLog','QUAL_naturalLog', 'VD_naturalLog', 'AF'], axis = 1)

,DP,VD,HIAF,IMPACT,SYMBOL,sampleTimePt,gender,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,chipOrControl
0,2036,6,0.0044,0.0,0.0,0.0,0.0,1.0,0.957230,1.000000,35.0,1.044670,0.0
5,2036,7,0.0035,1.0,0.0,0.0,0.0,1.0,0.988224,0.750000,35.3,1.317450,0.0
9,2040,5,0.0021,1.0,0.0,0.0,0.0,1.0,0.988166,0.666667,31.8,1.481960,0.0
13,2041,6,0.0025,1.0,0.0,0.0,0.0,1.0,0.983415,1.000000,32.7,1.016860,0.0
17,2037,5,0.0028,1.0,0.0,0.0,0.0,1.0,0.894942,0.666667,32.2,1.342210,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
692971,2130,9,0.0038,1.0,31.0,1.0,1.0,1.0,0.915913,1.250000,34.1,1.364554,1.0
692986,2076,6,0.0029,1.0,31.0,1.0,1.0,0.0,0.824007,0.000000,35.4,0.000000,1.0
692993,2069,7,0.0035,1.0,31.0,1.0,1.0,0.0,0.747064,0.000000,35.9,0.000000,1.0
693021,4805,13,0.0027,1.0,31.0,1.0,1.0,1.0,1.000839,0.857143,35.0,1.167610,1.0


## Models - Run 1


In [69]:
X = df1.drop('chipOrControl', axis = 1) # drop the target variable for the features
y = df1['chipOrControl'] # create a target dataframe
param_grid = {'n_estimators': [40, 50, 60], 'min_samples_split': [40, 50, 60, 70], 'min_samples_leaf': [12, 13, 14, 15, 16, 17], 
              'max_features': ['auto'], 'max_depth': [3, 4, 5, 6], 'criterion': ['gini'], 'bootstrap': [False]}

Neighbors

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state = 0)

knn = KNeighborsClassifier(n_neighbors = 3, weights= 'distance', metric = 'manhattan', algorithm = 'kd_tree')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))




Accuracy 0.811


RFC

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state = 0)

clf = RandomForestClassifier(bootstrap = True, ccp_alpha = 0.0, class_weight = None, criterion = 'gini',
                             max_depth = None, max_features = 0.262004613571483, max_leaf_nodes = None, 
                             #max_leaf_nodes = None,
                             max_samples = None, min_impurity_decrease = 0.0, 
                             min_impurity_split = 0.16768831357340783, min_samples_leaf =  1,
                             min_samples_split =0.16768831357340783,
                             min_weight_fraction_leaf = 0.0, n_estimators = 153, n_jobs = None, oob_score = False,
                             random_state = None, verbose = 0, warm_start = False)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.567


ADA Boost

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state = 0)

clf = AdaBoostClassifier(n_estimators=100, learning_rate =0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.565


Extra Trees Classifier

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)
etr = ExtraTreesClassifier()
etr.fit(X_train, y_train)
y_pred = etr.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.617


SGD Classifier

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)

sgd = SGDClassifier(loss = "squared_loss", early_stopping=True)
sgd.fit(X, y)
y_pred = sgd.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.567


Decision Tree

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0)

# instantatiate the RFC with 100 ensemble members
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
Y_pred = dtc.predict(X_test)  
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, Y_pred),3)))

Accuracy 0.699


## Models - Run 2


In [78]:
X = df2.drop('chipOrControl', axis = 1) # drop the target variable for the features
y = df2['chipOrControl'] # create a target dataframe

Neighbors

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state = 0)

knn = KNeighborsClassifier(n_neighbors = 3, weights= 'distance', metric = 'manhattan', algorithm = 'kd_tree')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))




Accuracy 0.698


RFC

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state = 0)

clf = RandomForestClassifier(bootstrap = True, ccp_alpha = 0.0, class_weight = None, criterion = 'gini',
                             max_depth = None, max_features = 0.262004613571483, max_leaf_nodes = None, 
                             #max_leaf_nodes = None,
                             max_samples = None, min_impurity_decrease = 0.0, 
                             min_impurity_split = 0.16768831357340783, min_samples_leaf =  1,
                             min_samples_split =0.16768831357340783,
                             min_weight_fraction_leaf = 0.0, n_estimators = 153, n_jobs = None, oob_score = False,
                             random_state = None, verbose = 0, warm_start = False)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.542


ADA Boost

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state = 0)

clf = AdaBoostClassifier(n_estimators=100, learning_rate =0.5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.533


Extra Trees Classifier

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)
etr = ExtraTreesClassifier()
etr.fit(X_train, y_train)
y_pred = etr.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.618


SGD Classifier

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)

sgd = SGDClassifier(loss = "squared_loss", early_stopping=True)
sgd.fit(X, y)
y_pred = sgd.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, y_pred),3)))

Accuracy 0.475


Decision Tree

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0)

# instantatiate the RFC with 100 ensemble members
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
Y_pred = dtc.predict(X_test)  
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, Y_pred),3)))

Accuracy 0.676
